Preparação

In [ ]:
pip install gcsfs

In [ ]:
pip install pandera

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 KB 3.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import os
import numpy as np
from google.cloud import storage
import pandera as pa

In [ ]:
pd.set_option('display.max_columns',100)

In [ ]:
serviceAccount = '/content/steady-citron-377820-c98e560167a8.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
client = storage.Client()
bucket = client.get_bucket('clerio-fernandes1')
bucket.blob('amsterdam_weekdays.cs')
path =  'gs://clerio-fernandes1/brutos/amsterdam_weekdays.csv'

In [ ]:
df = pd.read_csv(path, sep = ',')

Pre-analise

In [ ]:
df

,Unnamed: 0,realSum,room_type,room_shared,room_private,person_capacity,host_is_superhost,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat
0,0,194.033698,Private room,False,True,2.0,False,1,0,10.0,93.0,1,5.022964,2.539380,78.690379,4.166708,98.253896,6.846473,4.90569,52.41772
1,1,344.245776,Private room,False,True,4.0,False,0,0,8.0,85.0,1,0.488389,0.239404,631.176378,33.421209,837.280757,58.342928,4.90005,52.37432
2,2,264.101422,Private room,False,True,2.0,False,0,1,9.0,87.0,1,5.748312,3.651621,75.275877,3.985908,95.386955,6.646700,4.97512,52.36103
3,3,433.529398,Private room,False,True,4.0,False,0,1,9.0,90.0,2,0.384862,0.439876,493.272534,26.119108,875.033098,60.973565,4.89417,52.37663
4,4,485.552926,Private room,False,True,2.0,True,0,0,10.0,98.0,1,0.544738,0.318693,552.830324,29.272733,815.305740,56.811677,4.90051,52.37508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098,1098,2486.115342,Entire home/apt,False,False,2.0,False,0,0,10.0,100.0,1,2.375833,1.436054,181.198355,9.594573,225.304203,15.699521,4.85869,52.37677
1099,1099,233.637194,Private room,False,True,2.0,False,1,0,10.0,90.0,1,4.719736,0.322263,108.024880,5.719989,134.775277,9.391335,4.83611,52.34910
1100,1100,317.062311,Private room,False,True,2.0,False,1,0,10.0,92.0,1,0.598355,0.751993,440.476515,23.323524,625.947562,43.616927,4.88897,52.37798
1101,1101,1812.855904,Entire home/apt,False,False,4.0,False,0,0,8.0,84.0,5,1.943020,0.388532,257.948105,13.658524,336.589237,23.454022,4.90688,52.35794


In [ ]:
df.dtypes

Unnamed: 0                      int64
realSum                       float64
room_type                      object
room_shared                      bool
room_private                     bool
person_capacity               float64
host_is_superhost                bool
multi                           int64
biz                             int64
cleanliness_rating            float64
guest_satisfaction_overall    float64
bedrooms                        int64
dist                          float64
metro_dist                    float64
attr_index                    float64
attr_index_norm               float64
rest_index                    float64
rest_index_norm               float64
lng                           float64
lat                           float64
dtype: object

In [ ]:
dfbackup1= df.copy()

In [ ]:
df.head(2)

,Unnamed: 0,realSum,tipo_quarto,capacidade,anfitrião,nota_limpeza,rating,banheiros,lng,lat
0,0,194.033698,quarto_privado,2.0,não,10.0,93.0,1,4.90569,52.41772
1,1,344.245776,quarto_privado,4.0,não,8.0,85.0,1,4.90005,52.37432


Traducao

In [ ]:
df.drop(['room_shared','room_private','multi','biz','dist','metro_dist','attr_index', 'attr_index_norm','rest_index','rest_index_norm'], axis=1,inplace=True)

In [ ]:
df.rename(columns={'room_type':'tipo_quarto','person_capacity':'capacidade','host_is_superhost':'anfitrião','cleanliness_rating':'nota_limpeza','guest_satisfaction_overall':'rating','bedrooms':'banheiros'},inplace=True)

In [ ]:
pd.unique(df['tipo_quarto'])

array(['quarto_privado', 'Entire home/apt', 'Shared room'], dtype=object)

In [ ]:
df.loc[df.tipo_quarto == 'Private room',['tipo_quarto']] = 'quarto_privado'
df.loc[df.tipo_quarto == 'Entire home/apt',['tipo_quarto']] = 'casa_completa'
df.loc[df.tipo_quarto == 'Shared room',['tipo_quarto']] = 'quarto_compartilhado'

In [ ]:
df.loc[df.anfitrião == False , ['anfitrião']] = 'não'
df.loc[df.anfitrião == True , ['anfitrião']] = 'sim'

In [ ]:
df.isna().sum()

Unnamed: 0      0
realSum         0
tipo_quarto     0
capacidade      0
anfitrião       0
nota_limpeza    0
rating          0
banheiros       0
lng             0
lat             0
dtype: int64

In [ ]:
df.dtypes

Unnamed: 0        int64
realSum         float64
tipo_quarto      object
capacidade      float64
anfitrião        object
nota_limpeza    float64
rating          float64
banheiros         int64
lng             float64
lat             float64
dtype: object

In [ ]:
df['capacidade'] = df['capacidade'].astype(str)

In [ ]:
df.dtypes

Unnamed: 0        int64
realSum         float64
tipo_quarto      object
capacidade       object
anfitrião        object
nota_limpeza    float64
rating          float64
banheiros         int64
lng             float64
lat             float64
dtype: object

In [ ]:
schema = pa.DataFrameSchema(
    columns = {
        'tipo_quarto': pa.Column(pa.String),
        'anfitrião': pa.Column(pa.String, pa.Check.str_length(3, 3)),
        'nota_limpeza': pa.Column(pa.Float),
        'rating': pa.Column(pa.Float),
        'banheiros' : pa.Column(pa.Int),
        'capacidade': pa.Column(pa.String, pa.Check.str_matches(r'^[0-9]{2}\.[0]$'))
    }
)

Análise das variaveis

In [ ]:
df_copia.corr()

In [ ]:
df_copia.describe() #estatisticas dentro de cada coluna ou acessar direto a coluna [nome].describe

Filtro

In [ ]:

filtro = (df_copia['preço total'] > 100) & (df_copia['satisfação'] >= 70)
df_filtrado = df_copia.loc[filtro, ['preço total', 'satisfação']]